### Section bodies combined by score order top 1000 words

#### Steps:

1. Get the sections which have the scores greater than cutoff 0.744291
2. Extract the body of sections
3. Normalize the scores and adjusted number of words to be picked from the section
4. Maximize the word allocation in case some sections doesn't have required word
4. Extract top k words
5. If no relevant sections -> Write the complete file as summary with top 1000 words
6. Compare the system summary with each gold summary and get the rouge score. Take average of all gold summaries for the respective file.
7. Take average of all the scores for all the files.

In [32]:
import pandas as pd
import pickle
import os
import numpy as np

from extract_section_body import extract_section_body
from rouge_evaluation import get_rouge_scores
from maximal_word_allocation import get_number_of_words

In [33]:
VALIDATION_DATASET = True
TEST_DATASET = False

In [43]:
if VALIDATION_DATASET:
    dir_ = "../../../Dataset/FNS2023_Datasets/English/validation/"
    toc_loc_pkl_file_path = "../../../Dataset/FNS2023_Datasets/English/validation/out/valid_toc_loc.pkl"
    df_predicted_path = "../../../../DiMSum_FNP_2022/2_Section_Classification/out/validation_df_predicted.pkl"

if TEST_DATASET:
    dir_ = '../../../Dataset/FNS2022/English/testing/'
    toc_loc_pkl_file_path = '../../../Dataset/Annotated_Dataset/test_toc_loc.pkl'
    df_predicted_path = '../../FNP2022/2_Section_Classification/out/test_df_predicted.pkl'

annual_reports_dir = "annual_reports"
gold_summary_dir = "gold_summaries"
system_summary_dir = 'GPT_Summaries_Few_Shot'
team_name = 'SSC_AI_RG'
dir_

'../../../Dataset/FNS2023_Datasets/English/validation/'

In [44]:
df_predicted = pickle.load(open(df_predicted_path, 'rb'))
df_predicted

,file_id,toc_section,toc_section_pos,toc_section_len,is_section_in_summary,toc_section_cleaned,pred,False,True
0,30777,Financial and operational highlights,161,22,0,financi oper highlight,1,0.412114,0.587886
1,30777,Strategic report,183,6,1,strateg report,0,0.932721,0.067279
2,30777,Global network,189,11,0,global network,0,0.748624,0.251376
3,30777,Chairman’s statement,200,4,1,chairman statement,1,0.018934,0.981066
4,30777,Chief Executive’s review,204,4,1,chief execut review,1,0.006269,0.993731
...,...,...,...,...,...,...,...,...,...
9592,33155,Audit Committee report,6247,345,0,audit committe report,0,0.975820,0.024180
9593,33155,Directors’ Remuneration report,6592,1406,0,director remuner report,0,0.998092,0.001908
9594,33155,Corporate Governance Code,7998,69,1,corpor govern code,0,0.986258,0.013742
9595,33155,Directors’ biographies,8067,570,0,director biographi,0,0.991681,0.008319


In [45]:
def get_relevant_sections_with_score(file_id):
    cutoff_score = 0.744291
    df_dict = df_predicted[df_predicted.file_id == int(file_id)][['toc_section', 'True']].to_dict('list')
    section_score_dict = {}
    toc_sections = df_dict['toc_section']
    section_scores = df_dict['True']
    for i in range(len(toc_sections)):
        if section_scores[i] >= cutoff_score:
            section_score_dict[toc_sections[i]] = section_scores[i]
    return section_score_dict

In [46]:
def get_relevant_sections_with_body_len(file_id):
    section_body_len_dict = {}
    section_score_dict = get_relevant_sections_with_score(file_id)
    for section in section_score_dict.keys():
        body = extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
        section_body_len_dict[section] = len(body.split(' '))
    return section_body_len_dict

In [47]:
def get_section_number_of_words(file_id):
    section_num_words_dict = {}
    section_score_dict = get_relevant_sections_with_score(file_id)
    sections = list(section_score_dict.keys())
    section_scores = np.array(list(section_score_dict.values()))
    section_body_len_dict = get_relevant_sections_with_body_len(file_id)
    section_body_len = np.array(list(section_body_len_dict.values()))
    prev_num_required_words = np.zeros(len(section_body_len))
    num_words = get_number_of_words(section_scores, section_body_len, 1000, prev_num_required_words)
    for i in range(len(sections)):
        section_num_words_dict[sections[i]] = int(num_words[i])
    return section_num_words_dict

In [ ]:
num_file = 0
# Create a directory for the system-generated summaries
# os.makedirs(system_summary_dir)

# Iterate through files in the annual_reports_dir
for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        print("Processing File Number: ", num_file)
        num_file = num_file + 1
        
        # Extract the file_id from the filename
        file_id = file.split('.')[0]
        
        # Get relevant sections and their scores for the file
        relevant_sections_with_score = get_relevant_sections_with_score(file_id)
        
        # Section order is maintained
        relevant_sections = list(relevant_sections_with_score.keys())
        
        # Get the number of words in each section
        section_num_words_dict = get_section_number_of_words(file_id)
        
        summary = ""
        total_number_of_words_in_body = 0
        total_number_of_words_in_summary = 0
        
        print(file_id, relevant_sections, section_num_words_dict)
        print(file_id)
        if relevant_sections:
            print('Relevant Section Found in ', file_id)
            
            # Iterate through relevant sections
            for section in relevant_sections:
                number_of_words_to_be_extracted = section_num_words_dict[section]
                
                # Extract the body of the section
                section_body = extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
                section_body_split = section_body.split(' ')
                print(section_body_split)
                number_of_words_in_body = len(section_body_split)
                total_number_of_words_in_body = total_number_of_words_in_body + number_of_words_in_body
                
                # Generate the summary based on word limits
                if number_of_words_in_body > number_of_words_to_be_extracted:
                    summary = summary + " ".join(section_body_split[:number_of_words_to_be_extracted])
                    total_number_of_words_in_summary = total_number_of_words_in_summary + number_of_words_to_be_extracted
                else:
                    print(file_id, section, number_of_words_in_body, number_of_words_to_be_extracted)
                    summary = summary + " ".join(section_body_split[:number_of_words_in_body])
                    total_number_of_words_in_summary = total_number_of_words_in_summary + number_of_words_in_body
            
            print(file_id, 'number_of_words_in_output_summary', total_number_of_words_in_summary)    
            print(file_id, 'number_of_words_in_body', total_number_of_words_in_body)
            print('\n')
        else:
            print('Relevant Section Not Found in ', file_id)
            
            # Read the entire summary if no relevant sections are found
            summary = open(os.path.join(dir_, annual_reports_dir, file), "r", encoding="utf-8").read()
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            
            # Generate a shorter summary if the original summary is too long
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
        
        # Write the generated summary to a text file
        with open(os.path.join(system_summary_dir, file_id + '_' + team_name + '.txt'), 'w', encoding='utf-8') as f:
            f.write(str(summary))
        
        # Skip processing if the file is ".DS_Store"
        if ".DS_Store" in file:
            continue
    except Exception as e:
        print(file, e)


In [ ]:
if VALIDATION_DATASET:
    gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)
    rouge_scores = get_rouge_scores(system_summary_dir, gold_summary_dir_)
    rouge_scores

In [ ]:
rouge_scores

#### Langchain Implementation

##### Short note
- get_relevant_sections_with_body_len
- get_section_number_of_words
- get_relevant_sections_with_score
- We are using all the aboove functions to retrieve the necessary parameters to send as input to GPT



In [48]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import AzureChatOpenAI

import os
import pandas as pd

In [49]:
USE_GPT_3_5_TURBO = False
USE_GPT_4 = True

# Access - Config 


# Assign model 
if USE_GPT_4:
    model = AzureChatOpenAI(temperature=0,deployment_name="ss-gpt-32k")
elif USE_GPT_3_5_TURBO:
    model = AzureChatOpenAI(temperature=0,deployment_name="ss-gpt")
else:
    raise Exception('Model not supported.')
    
model

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='be51f10009fa41258fcd750a2fba07f2', openai_api_base='https://openai-ss.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='ss-gpt-32k', openai_api_type='azure', openai_api_version='2023-03-15-preview')

In [52]:
# Updated response schema
response_schemas = [
    ResponseSchema(name="Summary", description="summary of the section name from the section content around specified words"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [117]:
## zero shot prompt tempelate
Few_shot_template = """
The task is to summarize the section "{section_name}" with the following content: "{section_content}"

Here is the summary example : {summary_example}

Instructions:
- Create a summary of approximately {number_of_words} words.
- Ensure that the output language matches the input language.
- The summary should be a coherent and complete paragraph.

Format Instructions:
{format_instructions}
"""

In [118]:
def build_prompt():
    format_instructions = output_parser.get_format_instructions()
    
    print('Few Shot Setting')
    prompt_template = Few_shot_template
    input_vars = ["section_name", "section_content","number_of_words","summary_example"]
        
    prompt = ChatPromptTemplate(
    messages=[
            HumanMessagePromptTemplate.from_template(prompt_template)
    ],
        input_variables=input_vars,
        partial_variables={"format_instructions": format_instructions}
    )
    return prompt

In [119]:
prompt = build_prompt()

Few Shot Setting


In [122]:
def extract_summary(section_name, section_content, number_of_words, summary_example, prompt):
    # Format the input variables into the prompt
    _input = prompt.format_prompt(section_name=section_name, section_content=section_content, number_of_words=number_of_words, summary_example = summary_example)
    
    # Generate model output using the formatted input
    output = model(_input.to_messages())
    
    # Parse the model's response using the output_parser
    response = output_parser.parse(output.content)
    
    # Return the parsed response (summary)
    return response


In [136]:
import pandas as pd

# Read the Excel file into a DataFrame
few_shot_df = pd.read_excel("Few_Shot_Examples.xlsx", header = None)

# Initialize an empty dictionary
row_dict = {}

# Iterate through each row in the DataFrame
for index, row in few_shot_df.iterrows():
    key = None
    content = []
    for i, value in enumerate(row):
        if pd.notna(value):
            if key is None:
                key = value
            else:
                content.append(value)
    if key is not None:
        row_dict[key] = content

# Print the resulting dictionary
print(row_dict.keys())


dict_keys([30777, 30778, 30779, 30781, 30782, 30783, 30785, 30790, 30795, 30796, 30800, 30808, 30813, 30815, 30816])


In [159]:
row_dict[]

{30777: ['Chairman’s statement :  Bodycote plc annual report for the year ended 31 December 2017 25695    19 March 2018 3:29 PM    Proof 7\nThe automotive business saw growth of 14% with the majority \nof this increase coming from the car and light truck sector. This \ncompares to background demand growth in Europe of low single \ndigits and a slight decline in the USA. Our strong performance \nbenefited from the contribution from the acquisitions made in 2016, \ntogether with the investments we have made in Emerging Markets \nand Specialist T echnologies. ',
  'Chief Executive’s review : “2017 has once again demonstrated the quality of Bodycote’s business. \nThe Group’s revenue growth, combined with continued discipline on \ncosts, helped lift headline operating profit by 24%. Return on sales \nincreased to 18.0% from 16.6%. ”\nS. C. Harris'],
 30778: ["Highlights:\nIn 2017, Staffline Group achieved significant growth with sales increasing by 9% to £957.8 million. The revenue mix saw 

In [157]:
import pandas as pd

# Create an empty list to store the rows
summary_data = []
rouge_score_list = []
os.makedirs(system_summary_dir)
num_files_to_process = 300  # Change this to the desired number of files to process

num_file = 0

# Iterate through files in the annual_reports_dir
for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        if num_file >= num_files_to_process:
            break
            
        num_file = num_file + 1
        
        
        print("Processing File Number: ", num_file)
        
        
        # Extract the file_id and file_name from the filename
        file_id = file.split('.')[0]
        file_name = file
        if int(file_id) not in row_dict.keys():
            print("File id not available", file_id)
            break
        # Get relevant sections and their scores for the file
        relevant_sections_with_score = get_relevant_sections_with_score(file_id)
        
        # Section order is maintained
        relevant_sections = list(relevant_sections_with_score.keys())
        
        # Get the number of words in each section
        section_num_words_dict = get_section_number_of_words(file_id)
        
        summary = ""

        
        if relevant_sections:
    
            # Iterate through relevant sections
            for ind,section in enumerate(relevant_sections):
                
                if 'start' in section:
                    continue
            
                number_of_words_to_be_extracted = section_num_words_dict[section]
                section_body = extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
                summary_example = row_dict[int(file_id)][ind]
                summary_resp = extract_summary(section, section_body, number_of_words_to_be_extracted,summary_example, prompt)
                summary = summary + summary_resp['Summary']
                
                # Create a dictionary for the row and add it to the list
                row = {
                    'file_id': file_id,
                    'file_name': file_name,
                    'narrative_section_name': section,
                    'narrative_section_body': section_body,
                    'num_words_to_be_extracted': number_of_words_to_be_extracted,
                    'summary_generated': summary_resp['Summary']
                }
                summary_data.append(row)
            
        else:
            print('Relevant Section Not Found in ', file_id)
            
            # Read the entire summary if no relevant sections are found
            summary = open(os.path.join(dir_, annual_reports_dir, file), "r", encoding="utf-8").read()
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            
            # Generate a shorter summary if the original summary is too long
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
            
            # Create a dictionary for the row and add it to the list
            row = {
                'file_id': file_id,
                'file_name': file_name,
                'narrative_section_name': '',
                'narrative_section_body': '',
                'num_words_to_be_extracted': '',
                'summary_generated': summary
            }
            summary_data.append(row)
        
        with open(os.path.join(system_summary_dir, file_id + '_' + team_name + '.txt'), 'w', encoding='utf-8') as f:
            f.write(str(summary))
            if relevant_sections:
                row = {
                    'file_id': file_id,
                    'file_name': file_name,
                    'narrative_section_name': 'Final Summary',
                    'narrative_section_body': '',
                    'num_words_to_be_extracted': '',
                    'summary_generated': summary
                }
                summary_data.append(row)
            
        # Skip processing if the file is ".DS_Store"
        if ".DS_Store" in file:
            continue
        
    except Exception as e:
        print(file, e)
        
    if num_file % 2 == 0 : 
            if VALIDATION_DATASET:
                gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)
                rouge_scores = get_rouge_scores(system_summary_dir, gold_summary_dir_)
                print(rouge_scores)
                rouge_score_list.append(rouge_scores)

summary_df = pd.DataFrame(summary_data)

summary_df

Processing File Number:  1
Processing File Number:  2
Processing File Number:  0
Number of files processed:  2
{'rouge-1': {'p': 0.6773451582797336, 'r': 0.20345924581858582, 'f': 0.28454050585450363}, 'rouge-2': {'p': 0.2554551905109242, 'r': 0.09038147716431941, 'f': 0.12398918831941988}}
Processing File Number:  3
Processing File Number:  4
Processing File Number:  0
Number of files processed:  4
{'rouge-1': {'p': 0.5959774629859933, 'r': 0.3517536166994364, 'f': 0.3615756828840175}, 'rouge-2': {'p': 0.2482643509315206, 'r': 0.19020600906751806, 'f': 0.1787581413625704}}
Processing File Number:  5
Processing File Number:  6
Processing File Number:  0
Number of files processed:  6
{'rouge-1': {'p': 0.5778880656186584, 'r': 0.3144358663333858, 'f': 0.33720066194117915}, 'rouge-2': {'p': 0.20868966017295718, 'r': 0.1448026465492697, 'f': 0.14092352820464102}}
Processing File Number:  7
Processing File Number:  8
Processing File Number:  0
Number of files processed:  8
{'rouge-1': {'p':

,file_id,file_name,narrative_section_name,narrative_section_body,num_words_to_be_extracted,summary_generated
0,30777,30777.txt,Chairman’s statement,6 at 90%; £83.0m free cash flow 7 (2016: £60...,12,"Automotive business grew 14%, benefiting from ..."
1,30777,30777.txt,Chief Executive’s review,"■ ■ Full year ordinary dividend 17 .4p, up 10%...",57,"In the Chief Executive's review, S.C. Harris h..."
2,30777,30777.txt,Final Summary,,,"Automotive business grew 14%, benefiting from ..."
3,30778,30778.txt,highlights,p18 2 Staffline Group plc Annual Report 2017 ...,194,"In 2017, Staffline Group experienced significa..."
4,30778,30778.txt,Operational highlights,the two divisions for the last three years. 3...,252,The Recruitment division experienced significa...
5,30778,30778.txt,Chairman’s statement,£54m 6 Staffline Group plc Annual Report 2017 ...,277,"Staffline Group's Chairman, John Crabtree OBE,..."
6,30778,30778.txt,Chief Executive Officer’s statement,• To get the job done 10 Staffline Group plc A...,274,The CEO's statement for Staffline Group highli...
7,30778,30778.txt,Final Summary,,,"In 2017, Staffline Group experienced significa..."
8,30779,30779.txt,Highlights,119 Notes to the Consolidated Accounts 185 Co...,261,National Express Group PLC's 2017 financial hi...
9,30779,30779.txt,Chairman’s statement,Annual Report 2017 National Express Group PLC ...,257,The Chairman's statement in the Annual Report ...


In [158]:
if VALIDATION_DATASET:
    gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)
    rouge_scores = get_rouge_scores(system_summary_dir, gold_summary_dir_)
    print(rouge_scores)

Processing File Number:  0
Number of files processed:  14
{'rouge-1': {'p': 0.5676289382475165, 'r': 0.34350365754253503, 'f': 0.3495036529508936}, 'rouge-2': {'p': 0.20265498364344717, 'r': 0.14425764820975712, 'f': 0.13829994873053328}}


In [ ]:
rouge_score_list

In [ ]:
summar

In [ ]:
summary_df.to_csv('result.csv', index = False)